### Imports

In [14]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [15]:
# Carregar dados
data = pd.read_csv('C:\\Users\\Yuki\\Desktop\\github\\Aula_DS\\brasileirao_predicition\\data.csv')

# Certifique-se de que as colunas 'gols_mandante' e 'gols_visitante' existem
if 'gols_mandante' in data.columns and 'gols_visitante' in data.columns:
    data['gols_marcados_mandante'] = data.groupby('time_mandante')['gols_mandante'].expanding().mean().shift().reset_index(level=0, drop=True)
    data['gols_sofridos_mandante'] = data.groupby('time_mandante')['gols_visitante'].expanding().mean().shift().reset_index(level=0, drop=True)
    data['gols_marcados_visitante'] = data.groupby('time_visitante')['gols_visitante'].expanding().mean().shift().reset_index(level=0, drop=True)
    data['gols_sofridos_visitante'] = data.groupby('time_visitante')['gols_mandante'].expanding().mean().shift().reset_index(level=0, drop=True)
    data['diferenca_gols_mandante'] = data.groupby('time_mandante')['gols_mandante'].cumsum() - data.groupby('time_mandante')['gols_visitante'].cumsum()
    data['diferenca_gols_visitante'] = data.groupby('time_visitante')['gols_visitante'].cumsum() - data.groupby('time_visitante')['gols_mandante'].cumsum()

# Convertendo colunas categóricas para códigos numéricos
data['time_mandante'] = data['time_mandante'].astype('category').cat.codes
data['time_visitante'] = data['time_visitante'].astype('category').cat.codes

# Seleção de features e target
features = ['ano_campeonato', 'rodada', 'time_mandante', 'time_visitante', 'publico', 'gols_marcados_mandante', 'gols_sofridos_mandante', 'gols_marcados_visitante', 'gols_sofridos_visitante', 'diferenca_gols_mandante', 'diferenca_gols_visitante']
X = data[features]
y = (data['gols_mandante'] > data['gols_visitante']).astype(int) - (data['gols_mandante'] < data['gols_visitante']).astype(int)

# Mapeamento de y para classes 0, 1, 2
y = y.map({-1: 0, 0: 1, 1: 2})

# Divisão de dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definição do modelo e hiperparâmetros para RandomizedSearchCV
model = XGBClassifier(eval_metric='mlogloss', enable_categorical=True)  # Habilitando suporte a dados categóricos
param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma': [0, 0.1],
    'min_child_weight': [1, 5],
    'subsample': [0.7, 0.9], 'colsample_bytree': [0.7, 0.9] }

# Configuração do RandomizedSearchCV
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_grid,
    n_iter=100,  # Número de iterações
    cv=3,  # Redução de folds de validação cruzada
    verbose=1,
    n_jobs=-1  # Uso de todos os núcleos da CPU
)

#Treinamento
random_search.fit(X_train, y_train)

# Melhores parâmetros e desempenho
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Melhores parâmetros:", random_search.best_params_)
print("Acurácia:", accuracy_score(y_test, y_pred))

# Salvamento do modelo
with open('modelo_treinado.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print("Modelo salvo como 'modelo_treinado.pkl'.")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Melhores parâmetros: {'subsample': 0.7, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.9}
Acurácia: 0.5017772511848341
Modelo salvo como 'modelo_treinado.pkl'.
